In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [9]:
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

df_train = pd.DataFrame(titanic)

pd.set_option('display.max_rows', 4)  # Show 4 rows
pd.set_option('display.max_columns', None)  # Show all columns

# Dataset 1

Dropping     Cabin, Ticket, Name.

In [10]:
pd.set_option('display.max_rows', 4)  # Show all rows

df_train1 = df_train

df_train1.drop(columns = ['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

df_train1


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
...,...,...,...,...,...,...,...,...,...
889,890,1,1,male,26.0,0,0,30.0000,C
890,891,0,3,male,32.0,0,0,7.7500,Q


In [11]:
df_train1['Embarked'].fillna('S', inplace=True)

In [12]:
df_train1['Sex'] = df_train1['Sex'].map({'male': 1, 'female': 2})    # male=1 female=2

In [13]:
df_train1['Embarked'] = df_train1['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

df_train1   # Two NaN in Embarked column... index 61 829 both survived

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,1
1,2,1,1,2,38.0,1,0,71.2833,2
...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,2
890,891,0,3,1,32.0,0,0,7.7500,3


In [14]:
#Handling missing age

df_train1_age_Na = df_train1[df_train1['Age'].isna()]

df_train1 = df_train1.dropna(subset=['Age'])

In [15]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,1,NaN,0,0,8.4583,3
17,18,1,2,1,NaN,0,0,13.0000,1
...,...,...,...,...,...,...,...,...,...
878,879,0,3,1,NaN,0,0,7.8958,1
888,889,0,3,2,NaN,1,2,23.4500,1


In [16]:
df_train1_age = df_train1

df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']

/tmp/ipykernel_32/2467646141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']


In [17]:
df_train1

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,1,0,3,1,22.0,1,0,7.2500,1,1
1,2,1,1,2,38.0,1,0,71.2833,2,1
...,...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,2,0
890,891,0,3,1,32.0,0,0,7.7500,3,0


In [18]:
#Model to predict age.

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId', 'Survived'])
y_age = df_train1_age['Age']

scaler = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler.fit_transform(X_age)
y_age_scaled = scaler.fit_transform(y_age.values.reshape(-1, 1)).ravel()


X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

In [20]:
model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=28)

In [21]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForestRegressor instance
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

In [25]:
y_pred = best_rf.predict(X_test)

y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 9.147076376427146
Mean Squared Error: 141.1009020333041
R-squared: 0.3568452399753249


In [36]:
y_pred_round = np.round(y_pred)

array([[30.],
       [20.],
       [ 9.],
       [24.],
       [41.],
       [21.],
       [42.],
       [44.],
       [30.],
       [51.],
       [31.],
       [25.],
       [29.],
       [27.],
       [38.],
       [29.],
       [21.],
       [25.],
       [29.],
       [32.],
       [32.],
       [37.],
       [39.],
       [43.],
       [41.],
       [ 9.],
       [32.],
       [32.],
       [52.],
       [24.],
       [20.],
       [38.],
       [13.],
       [35.],
       [39.],
       [35.],
       [36.],
       [31.],
       [38.],
       [34.],
       [34.],
       [23.],
       [ 9.],
       [18.],
       [16.],
       [31.],
       [30.],
       [ 7.],
       [34.],
       [32.],
       [22.],
       [26.],
       [19.],
       [31.],
       [ 9.],
       [32.],
       [25.],
       [27.],
       [28.],
       [ 8.],
       [27.],
       [30.],
       [30.],
       [26.],
       [43.],
       [30.],
       [27.],
       [32.],
       [52.],
       [12.],
       [32.],
      

In [37]:
# now for the missing data.
pd.set_option('display.max_rows', 6) 
df_train1_age_Na


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
5,6,0,3,1,22.0,0,0,8.4583,3,0
17,18,1,2,1,35.0,0,0,13.0000,1,0
19,20,1,3,2,26.0,0,0,7.2250,2,0
...,...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,28.0,0,0,9.5000,1,0
878,879,0,3,1,24.0,0,0,7.8958,1,0
888,889,0,3,2,29.0,1,2,23.4500,1,3


In [38]:
df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']

/tmp/ipykernel_32/2457252628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']


In [39]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
5,6,0,3,1,22.0,0,0,8.4583,3,0
17,18,1,2,1,35.0,0,0,13.0000,1,0
19,20,1,3,2,26.0,0,0,7.2250,2,0
...,...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,28.0,0,0,9.5000,1,0
878,879,0,3,1,24.0,0,0,7.8958,1,0
888,889,0,3,2,29.0,1,2,23.4500,1,3


In [40]:
X_age = df_train1_age_Na.drop(columns = ['Age', 'PassengerId', 'Survived'])


X_age_scaled = scaler.fit_transform(X_age)

y_pred_na = best_rf.predict(X_age_scaled)


In [41]:
y_pred_na = y_pred_na.reshape(-1, 1)
y_pred_na = scaler_y.inverse_transform(y_pred_na)

In [42]:
y_pred_na

array([[22.28019565],
       [35.43711955],
       [25.58439923],
       [24.21751474],
       [19.73647338],
       [23.52711513],
       [31.67732475],
       [20.15863771],
       [24.21751474],
       [23.52711513],
       [22.89014313],
       [25.44991358],
       [20.15863771],
       [29.5564045 ],
       [38.18265376],
       [42.72733792],
       [22.01529416],
       [23.52711513],
       [22.89014313],
       [20.15863771],
       [22.89014313],
       [22.89014313],
       [23.52711513],
       [23.78237694],
       [22.93179952],
       [22.89014313],
       [22.66590941],
       [23.47778358],
       [28.68044994],
       [23.98633687],
       [23.32206122],
       [15.87749074],
       [26.06346668],
       [42.78943478],
       [14.21416109],
       [16.04557593],
       [34.41608855],
       [38.68481007],
       [22.72309122],
       [22.66590941],
       [20.15863771],
       [15.87749074],
       [21.23325287],
       [23.52711513],
       [13.73098548],
       [25

In [43]:
y_pred_na = np.round(y_pred_na)

In [44]:
df_train1_age_Na.loc[:, 'Age'] = y_pred_na


/tmp/ipykernel_32/2722028552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na.loc[:, 'Age'] = y_pred_na


In [45]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
5,6,0,3,1,22.0,0,0,8.4583,3,0
17,18,1,2,1,35.0,0,0,13.0000,1,0
19,20,1,3,2,26.0,0,0,7.2250,2,0
...,...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,28.0,0,0,9.5000,1,0
878,879,0,3,1,24.0,0,0,7.8958,1,0
888,889,0,3,2,29.0,1,2,23.4500,1,3


In [46]:
pd.set_option('display.max_rows', 10)

df_train1_age_Na['original_index'] = df_train1_age_Na.index
df_train1['original_index'] = df_train1.index

/tmp/ipykernel_32/2977192852.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na['original_index'] = df_train1_age_Na.index
/tmp/ipykernel_32/2977192852.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1['original_index'] = df_train1.index


In [47]:
merged_df = pd.concat([df_train1_age_Na, df_train1], axis=0)

In [48]:
merged_df.sort_values(by='original_index', inplace=True)

In [49]:
pd.set_option('display.max_rows', 10)

merged_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,original_index
0,1,0,3,1,22.0,1,0,7.2500,1,1,0
1,2,1,1,2,38.0,1,0,71.2833,2,1,1
2,3,1,3,2,26.0,0,0,7.9250,1,0,2
3,4,1,1,2,35.0,1,0,53.1000,1,1,3
4,5,0,3,1,35.0,0,0,8.0500,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,1,0,886
887,888,1,1,2,19.0,0,0,30.0000,1,0,887
888,889,0,3,2,29.0,1,2,23.4500,1,3,888
889,890,1,1,1,26.0,0,0,30.0000,2,0,889


In [50]:
df_version1 = merged_df.drop(columns = ['PassengerId', 'original_index'])

In [51]:
df_version1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,0,3,1,22.0,1,0,7.2500,1,1
1,1,1,2,38.0,1,0,71.2833,2,1
2,1,3,2,26.0,0,0,7.9250,1,0
3,1,1,2,35.0,1,0,53.1000,1,1
4,0,3,1,35.0,0,0,8.0500,1,0
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,1,0
887,1,1,2,19.0,0,0,30.0000,1,0
888,0,3,2,29.0,1,2,23.4500,1,3
889,1,1,1,26.0,0,0,30.0000,2,0


In [147]:
#Grid Search:
#Grid search exhaustively searches through a specified set of hyperparameter combinations. Here's how you can use grid search:

#Randomized Search:
#Randomized search explores a random subset of the hyperparameter space. This can be more efficient when the hyperparameter space is large.

In [52]:
X_v1 = df_version1.drop(columns = ['Survived'])
y_v1 = df_version1['Survived']

scaler_X = StandardScaler()
scaler_y = StandardScaler()
#scaler = MinMaxScaler()


X_v1 = scaler_X.fit_transform(X_v1)
y_v1 = scaler_y.fit_transform(y_v1.values.reshape(-1, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X_v1, y_v1, test_size = 0.2,random_state = 28)

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X_v1 = df_version1.drop(columns = ['Survived'])
y_v1 = df_version1['Survived']

scaler_X = StandardScaler()
X_v1 = scaler_X.fit_transform(X_v1)
X_train, X_test, y_train, y_test = train_test_split(X_v1, y_v1, test_size = 0.2,random_state = 28)


model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [54]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.79


In [55]:
y_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0])

In [56]:
y_test

287    0
621    1
395    0
273    0
155    0
      ..
405    0
220    1
585    1
817    0
55     1
Name: Survived, Length: 179, dtype: int64

# Prediction

In [58]:
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

df_test = pd.DataFrame(titanic_test)

In [59]:
pd.set_option('display.max_rows', None)  # Show 4 rows


df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.00,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.00,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.00,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.00,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.00,2,0,A/4 48871,24.1500,NaN,S


In [62]:
df_test_ageNa = df_test[df_test['Age'].isna()]

df_test_ageNa

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S
22,914,1,"Flegenheim, Mrs. Alfred (Antoinette)",female,NaN,0,0,PC 17598,31.6833,NaN,S
29,921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
36,928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,S
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S
41,933,1,"Franklin, Mr. Thomas Parham",male,NaN,0,0,113778,26.5500,D34,S
47,939,3,"Shaughnessy, Mr. Patrick",male,NaN,0,0,370374,7.7500,NaN,Q
54,946,2,"Mangiavacchi, Mr. Serafino Emilio",male,NaN,0,0,SC/A.3 2861,15.5792,NaN,C
58,950,3,"Davison, Mr. Thomas Henry",male,NaN,1,0,386525,16.1000,NaN,S


In [64]:
df_test_ageNa = df_test_ageNa.drop(columns = ['Cabin', 'Ticket', 'PassengerId', 'Name'])

df_test_ageNa['Family'] = df_test['SibSp'] + df_test['Parch']

KeyError: "['Cabin', 'Ticket', 'PassengerId', 'Name'] not found in axis"

Figuring out the age for test.csv

In [ ]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId'])
y_age = df_train1_age['Age']

scaler = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler.fit_transform(X_age)
y_age_scaled = scaler.fit_transform(y_age.values.reshape(-1, 1)).ravel()


X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForestRegressor instance
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

In [ ]:
y_pred = best_rf.predict(X_test)

y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)